# Work with latest OpenAI API version


In [ ]:
#!pip install openai

In [ ]:
#!pip install requests

In [ ]:
#!pip install num2words

In [ ]:
#!pip install pandas

In [ ]:
#!pip install openai.embeddings_utils

In [ ]:
#!pip install tiktoken

In [ ]:
import openai
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
#from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken

API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
#openai.api_version = "2023-05-15"
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")

#print(openai.api_base)
#print(openai.api_key)
url = openai.api_base  + "openai/models?api-version=2023-07-01-preview" 

r = requests.get(url, headers={"api-key": API_KEY})

#print(r.text)

In [ ]:
response = openai.chat.completions.create(
  model="gpt-35-turbo",
  #engine="gpt-35-turbo",
  messages = [{"role":"system","content":"You are an AI assistant that helps people find information."}],
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)

print(response.choices[0].message.content)

In [ ]:
response = openai.embeddings.create(
    input="Your text string goes here",
    #engine="text-embedding-ada-002"
    model="text-embedding-ada-002"
)
embeddings = response.data[0].embedding
print(embeddings[:5])

In [ ]:
df=pd.read_csv(os.path.join(os.getcwd(),'bill_sum_data.csv')) # This assumes that you have placed the bill_sum_data.csv in the same directory you are running Jupyter Notebooks
df

In [ ]:
df_CEqa=pd.read_csv(os.path.join(os.getcwd(),'智能Q&A_填寫部門_CE.csv')) # This assumes that you have placed the bill_sum_data.csv in the same directory you are running Jupyter Notebooks
df_CEqa

In [ ]:
df_bills = df[['text', 'summary', 'title']]
df_bills

In [ ]:
pd.options.mode.chained_assignment = None #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_bills['text']= df_bills["text"].apply(lambda x : normalize_text(x))

In [ ]:
df_CEqa['Question']=df_CEqa["Question"].apply(lambda x : normalize_text(x))

In [ ]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))
df_bills = df_bills[df_bills.n_tokens<8192]
len(df_bills)

In [ ]:
df_CEqa['n_tokens'] = df_CEqa["Question"].apply(lambda x: len(tokenizer.encode(x)))
df_CEqa = df_CEqa[df_CEqa.n_tokens < 8192]
len(df_CEqa)

In [ ]:
df_bills

In [ ]:
df_CEqa

In [ ]:
sample_encode = tokenizer.encode(df_bills.text[0]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
#decode

In [ ]:
ce_sample_encode = tokenizer.encode(df_CEqa.Question[0])
ce_decode = tokenizer.decode_tokens_bytes(ce_sample_encode)
ce_decode

In [ ]:
#from openai.embeddings_utils import get_embedding, cosine_similarity
def get_embedding(text, model="text-embedding-ada-002") :

    #Convert text to a word embedding
    text = text.replace('\n', ' ')
    return openai.embeddings.create(input = [text],model=model).data[0].embedding

In [ ]:
df_bills['ada_v2'] = df_bills["text"].apply(lambda x : get_embedding(x,  "text-embedding-ada-002"))

# engine = 'text-embedding-ada-002')) # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

In [ ]:
df_bills

In [ ]:
# engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
df_CEqa['ada_v2'] = df_CEqa["Question"].apply(lambda x : get_embedding(x,  'text-embedding-ada-002'))
df_CEqa

In [ ]:
df_bills

In [ ]:
def calculate_cosine_similarity(vector_a, vector_b):

    # Compute the dot product of the two vectors
    dot_product = np.dot(vector_a, vector_b)

    # Compute the L2 norms (magnitudes) of each vector
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)

    # Compute the cosine similarity
    # Note: We add a small epsilon value to the denominator for numerical stability
    epsilon = 1e-10
    cosine_similarity = dot_product / (norm_a * norm_b + epsilon)

    return cosine_similarity

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
# search through the reviews for a specific product
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        "text-embedding-ada-002" # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df_bills, "Can I get information on cable company tax revenue?", top_n=4)

In [ ]:
# search through the reviews for a specific product
def search_docs_CEQA(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        "text-embedding-ada-002" # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs (df_CEqa, "How can I find similar part?", top_n=4)

In [ ]:
print(res["CE Response"][res.index[0]])
print(res["Type"][res.index[0]])